In [33]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
NUMBER_OF_PARTICIPANTS = 12
DRAFT_NUM = 1
NUMBER_OF_PLAYERS = 13
COMPETITION_ID = 5
POOL_ID = 14
MY_ROSTER_ID = 21
CURRENT_TOURNAMENT_ROUND = 2
FINAL_ROUND_IN_TOURNAMENT = 7

In [2]:
from supabase import create_client, Client

In [3]:
supabase: Client = create_client(url, key)


In [4]:
def generate_autodraft_rankings(pool_id, draft_num):
    competition_id = supabase.table('pool').select('competition_id').eq('pool_id', pool_id).execute().data[0]['competition_id']
    print(competition_id)
    round_num = supabase.table('poolrule_draft').select('round_num').eq('pool_id', pool_id).eq('draft_num', draft_num).execute().data[0]['round_num']
    print(round_num)
    ranked_players = supabase.table('player_total_score_view').select('player_unique, total_points').eq('competition_id', competition_id).order('total_points', desc=True, nullsfirst=False).execute().data
    # print(ranked_players)
    available_players = [i['player_unique'] for i in supabase.table('view_available_players').select('player_unique').eq('pool_id', pool_id).execute().data]
    # rankings = [f"{i['player_unique']}-{i['total_points']}" for i in ranked_players if i['player_unique'] in available_players]
    rankings = [i['player_unique'] for i in ranked_players if i['player_unique'] in available_players]
    return rankings

In [35]:
print(generate_autodraft_rankings(POOL_ID, DRAFT_NUM))

5
2
[]


In [36]:
def generate_rankings_dict(pool_id = POOL_ID, draft_num: int = DRAFT_NUM, max_players: int = 1000, num_participants: int = NUMBER_OF_PARTICIPANTS):
    rostrankmap = {}
    draft_order = get_draft_order(draft_num=draft_num, pool_id=pool_id, num_participants=num_participants)
    rankings = supabase.table('draft_view').select('ranking, roster_id, player_unique').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(rankings)
    print(len(rankings.data))
    for row in rankings.data:
        if row['roster_id'] == MY_ROSTER_ID:
            print(row)

        if 'roster_id' not in row:
            continue
        roster_id = str(row['roster_id'])
        ranking = int(row['ranking'])
        player_unique = row['player_unique']
        if roster_id not in rostrankmap:
            rostrankmap[roster_id] = [None for i in range(0, max_players)]
            rostrankmap[roster_id][ranking] = player_unique
        else:
            rostrankmap[roster_id][ranking] = player_unique
    cleaned_map = {}
    for rost in rostrankmap:
        mapped_rankings = rostrankmap[rost]
        cleaned_map[str(rost)] = [i for i in mapped_rankings if i is not None]
    # for rost in draft_order:
    #     if rost not in cleaned_map:
    #         cleaned_map[str(rost)] = generate_autodraft_rankings(pool_id, draft_num)
    return cleaned_map

In [37]:
print(generate_rankings_dict())

data=[{'ranking': 1, 'roster_id': 21, 'player_unique': 'zach-edey-1'}, {'ranking': 2, 'roster_id': 21, 'player_unique': 'dalton-knecht-1'}, {'ranking': 3, 'roster_id': 21, 'player_unique': 'rj-davis-1'}, {'ranking': 4, 'roster_id': 21, 'player_unique': 'caleb-love-1'}, {'ranking': 5, 'roster_id': 21, 'player_unique': 'terrence-shannonjr-1'}, {'ranking': 6, 'roster_id': 21, 'player_unique': 'tristen-newton-1'}, {'ranking': 7, 'roster_id': 21, 'player_unique': 'cam-spencer-1'}, {'ranking': 8, 'roster_id': 21, 'player_unique': 'mark-sears-1'}, {'ranking': 9, 'roster_id': 21, 'player_unique': 'lj-cryer-1'}, {'ranking': 10, 'roster_id': 21, 'player_unique': 'antonio-reeves-1'}, {'ranking': 11, 'roster_id': 21, 'player_unique': 'alex-karaban-1'}, {'ranking': 12, 'roster_id': 21, 'player_unique': 'baylor-scheierman-1'}, {'ranking': 13, 'roster_id': 21, 'player_unique': 'trey-alexander-1'}, {'ranking': 14, 'roster_id': 21, 'player_unique': 'ryan-kalkbrenner-1'}, {'ranking': 15, 'roster_id': 21

In [21]:
def get_active_players(current_round: int = CURRENT_TOURNAMENT_ROUND, competition_id: int = COMPETITION_ID):
    available_players = supabase.table('view_active_players').select('player_unique, round_eliminated').eq('competition_id', competition_id).execute()
    return set([row['player_unique'] for row in available_players.data if row['round_eliminated'] == None])

In [13]:
print(get_active_players(current_round=CURRENT_TOURNAMENT_ROUND, competition_id=COMPETITION_ID))

{'ryan-schwarz-1', 'jameer-nelsonjr-1', 'aj-redd-1', 'mason-miller-1', 'justin-edwards-4', 'dylan-disu-1', 'elliot-cadeau-1', 'cade-phillips-1', 'zvonimir-ivisic-1', 'teo-rice-1', 'michel-ndayishimiye-1', 'townsend-tripple-2', 'alex-condon-1', 'kj-lewis-2', 'xavier-booker-2', 'julian-scott-3', 'steven-ashworth-1', 'elijah-tucker-1', 'blake-lampman-1', 'haston-alexander-1', 'keshon-gilbert-1', 'trey-stuart-1', 'james-scott-8', 'chayce-polynice-1', 'eddie-lampkinjr-1', 'jeff-woodward-1', 'nasir-mann-1', 'taviontae-jackson-1', 'hason-ward-1', 'richie-saunders-1', 'jack-francis-1', 'cade-bennett-1', 'kerwin-walton-1', 'dennis-parker-jr-1', 'jvonne-hadley-1', 'kobe-rodgers-1', 'javonte-johnson-1', 'johan-nziemi-1', 'myles-colvin-2', 'kyeron-lindsay-1', 'ileri-ayo-faleye-1', 'dj-wagner-1', 'tre-king-1', 'luke-murphy-3', 'dillon-wilhite-1', 'dj-horne-1', 'wesley-fields-1', 'manny-obaseki-1', 'zaire-williams-1', 'aubin-gateretse-1', 'tre-holloman-1', 'camden-heide-1', 'morris-ugusuk-1', 'javoh

# Draft

In [38]:
def run_draft(expected_participants: int = NUMBER_OF_PARTICIPANTS, round_start: int = CURRENT_TOURNAMENT_ROUND, round_end: int = FINAL_ROUND_IN_TOURNAMENT, pool_id: int = POOL_ID, draft_num: int = DRAFT_NUM, starting_pick_num: int = 1):
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(draft_rules)
    roster_count = draft_rules.data[0]['roster_count']
    round_num = draft_rules.data[0]['round_num']
    draft_time = draft_rules.data[0]['draft_time']
    draft_order = draft_rules.data[0]['draft_order']
    draft_order = get_draft_order(num_participants=expected_participants, pool_id=pool_id, draft_num=draft_num)
    rankings_dict = generate_rankings_dict(pool_id, draft_num)
    available_players_set = get_active_players()
    last_pick_dict = {str(key): 0 for key in draft_order}
    print(last_pick_dict)
    print(draft_order)
    pick_num = starting_pick_num
    draft_insert = []
    # print(rankings_dict)
    print(rankings_dict.keys())
    print(":::RANKINGS DICT:::", rankings_dict)
    print(available_players_set)
    print(last_pick_dict)
    print(":::DRAFT ORDER:::", draft_order)
    print("Starting Draft")
    for draft_round in range (0, roster_count):
        for round_pick in range (0, expected_participants):
            made_pick = False
            current_drafter = str(draft_order[round_pick])
            for pick in range (last_pick_dict[current_drafter], len(rankings_dict[current_drafter])):
                player_unique = rankings_dict[current_drafter][pick]
                if player_unique in available_players_set:
                    print(f'Player {player_unique} picked by {current_drafter} at pick {pick_num}')
                    last_pick_dict[current_drafter] = pick
                    available_players_set.remove(player_unique)
                    pick_row = {
                        'roster_id': int(current_drafter),
                        'player_unique': player_unique,
                        'round_start': round_start,
                        'round_end': round_end,
                        'pick_number': pick_num,
                        'draft_num': draft_num
                    }
                    draft_insert.append(pick_row)
                    made_pick = True
                    break
            if not made_pick:
                raise Exception(f'No pick made by {current_drafter} at pick {pick_num} round {draft_round+1}')
            pick_num += 1
            if round_pick == expected_participants - 1:
                print(f'Round {draft_round} complete')
                draft_order.reverse()
    print(draft_insert)
    return draft_insert
    
def record_draft_in_db(draft_insert):
    supabase.table('roster_player').upsert( draft_insert, ignore_duplicates=True ).execute()

In [43]:
draft_results = run_draft(round_start=CURRENT_TOURNAMENT_ROUND, draft_num=DRAFT_NUM, expected_participants=NUMBER_OF_PARTICIPANTS)

data=[{'draft_num': 1, 'pool_id': 14, 'roster_count': 13, 'draft_time': '2024-03-21T12:00:00', 'draft_order': 0, 'round_num': 2}] count=None
data=[{'ranking': 1, 'roster_id': 21, 'player_unique': 'zach-edey-1'}, {'ranking': 2, 'roster_id': 21, 'player_unique': 'dalton-knecht-1'}, {'ranking': 3, 'roster_id': 21, 'player_unique': 'rj-davis-1'}, {'ranking': 4, 'roster_id': 21, 'player_unique': 'caleb-love-1'}, {'ranking': 5, 'roster_id': 21, 'player_unique': 'terrence-shannonjr-1'}, {'ranking': 6, 'roster_id': 21, 'player_unique': 'tristen-newton-1'}, {'ranking': 7, 'roster_id': 21, 'player_unique': 'cam-spencer-1'}, {'ranking': 8, 'roster_id': 21, 'player_unique': 'mark-sears-1'}, {'ranking': 9, 'roster_id': 21, 'player_unique': 'lj-cryer-1'}, {'ranking': 10, 'roster_id': 21, 'player_unique': 'antonio-reeves-1'}, {'ranking': 11, 'roster_id': 21, 'player_unique': 'alex-karaban-1'}, {'ranking': 12, 'roster_id': 21, 'player_unique': 'baylor-scheierman-1'}, {'ranking': 13, 'roster_id': 21, '

In [42]:
print([i for i in range(0,12)])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [44]:
record_draft_in_db(draft_results)

In [17]:
import random
import datetime
import json
def generate_draft_order(pool_id: int = POOL_ID, draft_num: int = DRAFT_NUM):
    rosters = supabase.table('roster_full_view').select('roster_id, rankings_submitted').eq('pool_id', pool_id).execute()
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    draft_rule_order = draft_rules.data[0]['draft_order']
    roster_list = []
    if draft_num == 1 or draft_rules.data[0]['draft_order'] == 0:
        roster_list = [row['roster_id'] for row in rosters.data]
        random.shuffle(roster_list)
    else:
        prev_draft_order = [row['roster_id'] for row in supabase.table('roster_draftorder').select('roster_id').eq('draft_num', draft_num-1).execute().data]
        if draft_rule_order == 1:
            roster_list = prev_draft_order
        elif draft_rule_order == -1:
            prev_draft_order.reverse()
            roster_list = prev_draft_order
        print(roster_list)
    order_insert = []
    created_at = json.dumps(datetime.datetime.now(), indent=4, sort_keys=True, default=str)
    for idx, roster_id in enumerate(roster_list):
        row = {
            'draft_num': draft_num,
            'roster_id': roster_id,
            'created_at': created_at,
            'draft_order': idx+1
        }
        order_insert.append(row)
    supabase.table('roster_draftorder').upsert( order_insert, on_conflict='draft_num, roster_id', ignore_duplicates=False).execute()
    return roster_list

In [125]:
generate_draft_order(draft_num=2)

[11, 9, 13, 14, 16, 12, 17, 10, 18]

In [18]:
def get_draft_order(num_participants: int = NUMBER_OF_PARTICIPANTS, pool_id=POOL_ID, draft_num=DRAFT_NUM):
    draft_order_data = supabase.table('draft_order_view').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if len(draft_order_data.data) == num_participants:
        draft_order = [None for i in range(0, num_participants + 1)]
        for row in draft_order_data.data:
            draft_order[row['draft_order']] = row['roster_id']
    else:
        draft_order = generate_draft_order(pool_id, draft_num)
    return [i for i in draft_order if i is not None]

In [135]:
print(get_draft_order())

[18, 10, 17, 12, 16, 14, 13, 9, 11]


In [83]:
get_draft_info()

[{'draft_num': 1, 'pool_id': 8, 'roster_count': 12, 'draft_time': '2023-03-16T11:00:00', 'draft_order': 0, 'round_num': 2}]


In [74]:
generate_rankings_dict(pool_id=8)

data=[{'draft_num': 1, 'pool_id': 8, 'roster_count': 12, 'draft_time': '2023-03-16T11:00:00', 'draft_order': 0, 'round_num': 2}] count=None
{'ranking': 48, 'roster_id': 12, 'player_unique': 'jayden-gardner-1'}
{'ranking': 48, 'roster_id': 12, 'player_unique': 'jayden-gardner-1'}
{'ranking': 49, 'roster_id': 12, 'player_unique': 'mitchell-saxen-1'}
{'ranking': 49, 'roster_id': 12, 'player_unique': 'mitchell-saxen-1'}
{'ranking': 50, 'roster_id': 12, 'player_unique': 'anton-watson-1'}
{'ranking': 50, 'roster_id': 12, 'player_unique': 'anton-watson-1'}
{'ranking': 51, 'roster_id': 12, 'player_unique': 'fletcher-loyer-1'}
{'ranking': 51, 'roster_id': 12, 'player_unique': 'fletcher-loyer-1'}
{'ranking': 52, 'roster_id': 12, 'player_unique': 'kj-adamsjr-1'}
{'ranking': 52, 'roster_id': 12, 'player_unique': 'kj-adamsjr-1'}
{'ranking': 53, 'roster_id': 12, 'player_unique': 'jarace-walker-1'}
{'ranking': 53, 'roster_id': 12, 'player_unique': 'jarace-walker-1'}
{'ranking': 54, 'roster_id': 12, '

In [ ]:
def update_tables():

In [ ]:
def run_draft(round_start, round_end):
